In [317]:
import pandas as pd
from pandas import DataFrame, read_csv, DataFrame, Series
import numpy as np

In [318]:
# получение данных о студентах и их оценках по дисциплинам
import csv
def read_file(filename):
    f = open(filename)
    data = csv.reader(f)
    mentions = dict()
    for line in data:
        student = line[0]
        discipline = line[6]
        type_d = line[7]
        sem_rate = int(line[8])
        bonus = int(line[9])
        exam_rate = int(line[10])
        rate = 0
        if type_d == 'exam':
            if (sem_rate >= 38) & (exam_rate >= 22):
                rate = sem_rate + exam_rate + bonus
        else: 
            if (sem_rate >= 60):
                rate = sem_rate + bonus
        if not student in mentions:
            mentions[student] = dict()
        mentions[student][discipline] = rate
    f.close()
    return mentions

In [319]:
# косинусная мера сходства: насколько два пользователя похожи друг на друга. 
def dist_cosine(vecA, vecB):
    def dot_product(vecA, vecB):
        d = 0.0
        for dim in vecA:
            if dim in vecB: 
                d += vecA[dim]*vecB[dim]
        return d
    a = dot_product(vecA,vecB)
    b = math.sqrt(dot_product(vecA,vecA))
    c = math.sqrt(dot_product(vecB,vecB))
    if (a == 0) or (b == 0) or (c == 0):
        return 0
    else:
        return  (a / (b * c))

In [320]:
import math
def similar_students(studentID, studentsData, nBestStudents):
    matches = [(u, dist_cosine(studentsData[studentID], studentsData[u])) for u in studentsData.keys() if u != studentID]
    bestMatches = sorted(matches, key=lambda q:(q[1], q[0]), reverse=True)[:nBestStudents]
    print("Most correlated with studentID: " + str(studentID))
    for line in bestMatches:
        print("StudentID: " + str(line[0]) + "  Coeff: " + str(line[1])) 
    return [(x[0], x[1]) for x in bestMatches]

In [321]:
students_test = read_file("raw_data32.csv")
#df = DataFrame(students_test)
#df.columns.name = 'studentID'
#df.index.name = 'discipline'    
rec = similar_students('1', students_test, 10)


Most correlated with studentID: 1
StudentID: 9  Coeff: 0.7542152087381664
StudentID: 6  Coeff: 0.7248624347806895
StudentID: 7  Coeff: 0.7194365076928019
StudentID: 5  Coeff: 0.6837763094049188
StudentID: 4  Coeff: 0.5597650673256156
StudentID: 2  Coeff: 0.22271363971105235
StudentID: 3  Coeff: 0.06705017139158977
StudentID: 8  Coeff: 0


In [322]:
# Похожесть дисциплин
def read_subjects(filename):
    f = open(filename)
    data = csv.reader(f)
    subjects = dict()
    for line in data:
        ID = line[0]
        discipline = line[1]
        subjects[ID] = discipline
    f.close()
    return subjects

import sklearn.datasets
from sklearn.cluster import KMeans

#num_clusters = 50
#km = KMeans(n_clusters=num_clusters, init='random', n_init=1, verbose=1, random_state=3)
#km.fit(vectorized)
#data_subjects = read_subjects('subjects.csv')
#s = "hello world"
#r = vectorizer.transform([s])
#l = km.predict(r)[0]

In [327]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn import *


from pandas import DataFrame

sd = read_subjects("subjects.csv")

# nltk.download() 

from nltk.corpus import stopwords
stopwords = set(stopwords.words("russian"))
#print(stopwords)


In [328]:
def review_to_words( review_text ):
    review_text = review_text.replace('ё','е')
    review_text = review_text.replace('Ё','Е')
    # 1. Удаляем все, кроме букв
    letters_only = re.sub("[^а-яА-Я]", " ", review_text) 
    # 2. Делаем все буквы строчными и создаем массив слов
    words = letters_only.lower().split()                             
    # 3. Удаляем незначащие слова
    meaningful_words = [w for w in words if not w in stopwords]
    # 4. Формируем текст, объединяя слова через пробел
    return( " ".join( meaningful_words ))

In [336]:
d0 = pd.read_csv("subjects2.csv", encoding='utf-8')
#d0.head(3)

In [338]:
d1 = d0.copy()
d1['Name'] = d1['Name'].apply(lambda s: review_to_words(s))
#d1.head(3)

In [340]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None,\
                             max_features = 50000, binary = False) 
bagOfWords = vectorizer.fit_transform(d1['Name'])
bagOfWords = preprocessing.normalize(bagOfWords, norm='l1')
vocabulary = vectorizer.get_feature_names()

/home/anna/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [346]:
bagOfWordsTrain = bagOfWords[0:500,:]
bagOfWordsTest = bagOfWords[500:]

print(vocabulary)

['автоматизация', 'автоматов', 'агрессия', 'адаптация', 'адвокатура', 'административное', 'административному', 'администрирование', 'активных', 'актуальные', 'актуарная', 'алгебра', 'алгебраические', 'алгебраических', 'алгебраической', 'алгебру', 'алгебры', 'алгоритмов', 'алгоритмы', 'альтернативы', 'анализ', 'анализа', 'анализе', 'анализу', 'аналитик', 'аналитическая', 'анатомия', 'английский', 'ансис', 'антикоррупционная', 'антикоррупционного', 'антикризисного', 'аппарат', 'аппаратно', 'аппаратные', 'арбитражном', 'арбитражный', 'архитектура', 'архитектуры', 'асимптотические', 'аспекты', 'атмосферы', 'аэроакустика', 'аэрогидродинамика', 'аэрогидродинамике', 'базе', 'базовые', 'базы', 'банахова', 'банковские', 'банкротстве', 'барьеров', 'бд', 'безопасности', 'безопасность', 'бенчмаркинг', 'бжд', 'библиотека', 'бизнес', 'бизнеса', 'биоинформатики', 'биомеханика', 'биомеханике', 'биомеханики', 'биомеханику', 'бифуркации', 'бифуркаций', 'бумагами', 'бухгалтерия', 'бухгалтерские', 'бухгал